<a href="https://colab.research.google.com/github/AMerrington/sense-hackathon/blob/jacob/Patching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print ("Done 1")

Mounted at /content/drive
Done 1


In [14]:
import os
import numpy as np
import xarray as xr
!pip install rasterio
import rasterio
import cv2

In [12]:
"""
Importing the S1 images
"""

#from write_tiff_funcs_DTM import create_geoTrans
#from write_tiff_funcs_DTM import check_array_orientation
#from write_tiff_funcs_DTM import write_xarray_to_GeoTiff

# Setting directories
SAMPLING_DIR = "/content/drive/MyDrive/ATI_Hackathon/Sampling/"
META_DIR = "/content/drive/MyDrive/ATI_Hackathon/Sampling/"
S1_tiffs= "/content/drive/MyDrive/ATI_Hackathon/Sentinel_geotiffs/"
idx=0
raster=0
# Importing S1 tiffs.
tiffs = os.listdir(S1_tiffs)
print (type(tiffs), tiffs)
num_of_files = 0
for filename in [tiffs[0]]:
    # Opening tiffs as an xarray.
    raster= xr.open_rasterio('%s/%s' % (S1_tiffs, filename))
    print (f"Opened tiff file as raster.")
    n_bands= raster.values.shape[0]
    
    # THIS IS NOT REFLECTANCE but does rescale (normalise bands) from 0 to 1.
    #img= exposure.rescale_intensity(raster)
    
    # apply NaN values to empty cells in multispec image.
    for ii in range(n_bands):
        raster.values= raster.values.astype('float')
        raster.values[raster.values==raster.nodatavals[ii]]= np.nan
        #raster= raster.rio.reproject('EPSG:27700') # reprojecting to British National Grid for use in GIS.
    
    raster= raster.transpose('y','x','band') 

    # Tiling this S1 tiff.
    img_shape = raster.values.shape
    print (f"Image shape: {img_shape}")

    num_tiles= 100
    x_size= img_shape[1]/10
    y_size= img_shape[0]/10



    x_size=200
    y_size=200

    # xarray
    j= (0, 0)
    offset = (int(y_size), int(x_size))
    #segments= np.zeros(raster.values.shape[0:2])
    
    x_coords= raster.coords['x'].values
    y_coords= raster.coords['y'].values
    meta = []
    for i in range(1, num_tiles + 1): 
            for ii in range(1, num_tiles + 1): 
                try:
                    tile = raster.sel(x=slice(x_coords[j[0]], x_coords[offset[1]*i-1]), y=slice(y_coords[j[1]], y_coords[offset[0]*ii-1]))
                    # Writing out the tiled tiff
                    cv2.imwrite(SAMPLING_DIR + str(idx) + ".png", tile.values)
                    num_of_files += 1

                    tile="" # Maybe help with ram?
                    meta.append([idx, i, ii])
                    # Moving the offset to the next column (row keeps static).
                    j= (j[0], offset[1]*ii)
                    idx += 1
                except IndexError:
                    pass#print ('At edge of image')
            # Moving to the next row and resetting the column offsets.
            j= (offset[0]*i, 0)
            print('Tile segmented: '+ str(i*ii) +' of '+ str(num_tiles**2))
    raster="" # Maybe help with ram?
    np.save(SAMPLING_DIR + f'meta_{x_size}x{y_size}.npy', np.array(meta))
    print (f"Number of files output: {num_of_files}")

<class 'list'> ['S1A_EW_GRDM_1SDH_20180116T075430_20180116T075530_020177_0226B9_9FE3_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180312T074605_20180312T074705_020979_02403B_43FE_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180212T180416_20180212T180516_020577_02336E_5855_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180313T181225_20180313T181325_021000_0240E1_8163_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180417T074606_20180417T074706_021504_0250C3_D211_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180513T072951_20180513T073051_021883_025CBD_A70C_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180611T073807_20180611T073907_022306_026A0D_CC44_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180612T180423_20180612T180523_022327_026AB3_AC33_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180717T073809_20180717T073909_022831_0279B9_EBF1_Orb_Cal_Spk_TC_rgb_8bit.tif', 'S1A_EW_GRDM_1SDH_20180813T174720_20180813T174820_023231_028669_AA12_Orb_Cal_Spk_TC_rgb_8bit.